# MySQL with Python

## Overview 
### What You'll Learn
In this section you will learn: 
1. How to use MySQL with Python

### Prerequisites
Before starting this section, you will need: 
1. To know intermediate Python - [Python Workshop](https://github.com/HackBinghamton/PythonWorkshop)
2. To be familiar with basic SQL concepts and syntax - [Intro to SQL](SQLBeginner.md)

### Introduction

MySQL is a popular, open-source relational database management system backed by Oracle. You would need to use SQL, a query language, to communicate with the relational database. However, MySQL allows you to connect the SQL language with Python, a procedural language. In this section you will learn how to connect to a MySQL database and perform some SQL operations using Python.

## Setting Up The Environment

First, you will need to install the MySQL server. There is a free version of MySQL that you can install on your local machines if you'd like, but here, we can install it within this Google Colab notebook. 

Run the following code to install MySQL, and wait for the installation to complete. 

In [ ]:
!apt update
!apt install mysql-server

Now that you have MySQL installed, you can start the server.


In [ ]:
# !service mysql stop
!mysqld --skip-grant-tables &
!service mysql start

Normally, you would set up a user and a password to connect to the server, but since you are doing this within Google Colab, trying to log in with a password causes some connection issues. That is why we are calling `!mysqld --skip-grant-tables &` above; by running this command, it allows connections to all your databases without any passwords or restrictions. **Generally, you should avoid providing open access to your databases as it is a major security concern.**

The final step to set up your environment is to install the MySQL Python Connector. MySQL connector/Python is a database driver that allows you to access MySQL databses using Python. Run the following command to install the connector.

In [ ]:
!python3 -m pip install mysql-connector-python

## Python MySQL

### Connecting to the MySQL server

In order to connect to the server, you need to import the python module you installed above. Then, call the `connect()` constructor that takes in optional parameters: `user`, `password`, `host`, and `database`. In this Google Colab instance, the user is "root" and the host is "localhost" (or IP address of "127.0.0.1"). Since you do not need a password to connect to the server, and you have not created any databases yet, you can leave them out.The `connect()` constructor will return a `MySQLConnection` object.


In [ ]:
import mysql.connector

cnx = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
)

print(cnx)

### Creating a Database

Now with your connection, you can start executing SQL commands with Python. The first thing you should do is to make a database to work with. 

In order to execute commands, you need a cursor object. A cursor is a control structure that points to and traverses through your database. 

With your cursor object, you can call the `execute()` function and pass in SQL queries with normal MySQL syntax as a string. After executing the query, the server sends back data in which the cursor object acts as an iterator traversing through the returned data.

The follow code snippet creates a cursor object then creates a database called `my_db`. The next query retrieves the names of existing databases in your server. Calling the `execute()` function itself will not list out the databases. Since the cursor objects acts as an iterator to the returned data, the code loops through the cursor and prints out every record from the query -- all the names of the databases. 

In [ ]:
cursor = cnx.cursor(buffered=True)

cursor.execute("CREATE DATABASE IF NOT EXISTS my_db")

cursor.execute("SHOW DATABASES")

for x in cursor:
  print(x)


The above code snippet should print out the name of your new database `my_db` and the default databases the MySQL server manages. 

### Creating a Table

Now that you have a database to work with, you'll need to connect to the server again but this time specifying which database you want to connect to. The following code snippet will connect to the `my_db` database you created above. 

In [ ]:
cnx = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  database='my_db'
)
cursor = cnx.cursor(buffered=True)

The `my_db` database does not have any tables yet. The following code will create a table called `menu` that has two columns: `name` and `price`. 

In [ ]:
cursor.execute("CREATE TABLE IF NOT EXISTS menu (name VARCHAR(255), price DOUBLE)")

You can see if the table was created by listing out all the tables the database has.

In [ ]:
cursor.execute("SHOW TABLES")

for x in cursor:
  print(x)

### Insert Values to Table

The following code snippet will insert a row into the `menu` table. 

To insert values into your tables, you can use queries with parameters. With the `%s` marker, you can specify and pass in the values separately (similar to how `printf()` works, but all the parameter specifer is the `%s` symbol). 

When writing to a table, you must call the `commit()` method on your connection object to keep the changes. 

In [ ]:
sql = "INSERT INTO menu (name, price) VALUES (%s, %s)"
val = ("Steak", "29.99")
cursor.execute(sql, val)

cnx.commit()

cursor.execute("SELECT * FROM menu")

for x in cursor:
  print(x)

The above snippet should return an entry in your `menu` table. 

You can also use the `executemany()` method to execute multiple queries by passing a list of values like so:

In [ ]:
sql = "INSERT INTO menu (name, price) VALUES (%s, %s)"
val = [
       ("Spaghetti", "8.50"),
       ("Chicken Parmesan", "10.25"), 
       ("Fried Icecream", "3.99")
]

cursor.executemany(sql, val)

cnx.commit()

cursor.execute("SELECT * FROM menu")

for x in cursor:
  print(x)

### Selecting From Table

As you've seen, the cursor object iterates through the returned data from the last executed query. If you wanted to save your query results before calling a new one, you can use the `fetchall()` or `fetchone()` methods. 

The `fetchall()` method will return all the rows satisfied from your latest executed query; `fetchone()` will return the first row of the result. 

Run the following code snippet and see what prints out. 

In [ ]:
cursor.execute("SELECT * FROM menu WHERE price < 20.00")

store_results = cursor.fetchall()

cursor.execute("SELECT * FROM menu WHERE name LIKE 's%'" )

store_results2 = cursor.fetchone(); 

print("Printing result of first query")
print(store_results)

print("\n Printing first row of second query")
print(store_results2)

### Close Connections 

Once you're done with accessing your database and server, make sure to close these connections using the `close()` method. 

In [ ]:
cursor.close()
cnx.close()

## Exercise

Now that you learned how to create databases and tables, insert values into tables, and access the tables with Python and MySQL, continue playing around it. Try writing queries you've learned in the SQL portions of this workshop! (Reminder: MySQL and MS SQL might have slighlty different SQL syntax)

In [ ]:
# Your code here!